In [1]:
"""
以下是推理结果，有几个warning，但推理结果和torch一致
"""
from mindnlp.transformers import CogVLMConfig,MLP,RMSNorm,PretrainedConfig,LlamaTokenizer,AutoModelForCausalLM
import mindspore
mindspore.set_seed(42)
ms_net = AutoModelForCausalLM.from_pretrained(
    'THUDM/cogvlm-chat-hf'
)
from PIL import Image
tokenizer = LlamaTokenizer.from_pretrained('lmsys/vicuna-7b-v1.5')
query = 'describe this picture'
image = Image.open('CogVLM.png').convert('RGB')
ms_inputs = ms_net.build_conversation_input_ids(tokenizer, query=query, history=[],images=[image])  # chat mode

ms_inputs = {
    'input_ids': ms_inputs['input_ids'].unsqueeze(0),
    'token_type_ids': ms_inputs['token_type_ids'].unsqueeze(0),
    'attention_mask': ms_inputs['attention_mask'].unsqueeze(0),
    'images': [[mindspore.Tensor(ms_inputs['images'][0],dtype=mindspore.float16)]],
}

gen_kwargs = {"max_length": 2048, "do_sample": False}
ms_out = ms_net.generate(**ms_inputs,**gen_kwargs)
outputs = ms_out[:, ms_inputs['input_ids'].shape[1]:]
print(tokenizer.decode(outputs[0]))

In [6]:
query = '你好'
text_only_template = f"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {query} ASSISTANT:" 
query = text_only_template
image = Image.open('/hpc2hdd/home/ypeng455/mindnlp/mindnlp/transformers/models/cogvlm/CogVLM.png').convert('RGB')
ms_inputs = ms_net.build_conversation_input_ids(tokenizer, query=query, history=[]) # chat mode

ms_inputs = {
    'input_ids': ms_inputs['input_ids'].unsqueeze(0),
    'token_type_ids': ms_inputs['token_type_ids'].unsqueeze(0),
    'attention_mask': ms_inputs['attention_mask'].unsqueeze(0),
}

gen_kwargs = {"max_length": 2048, "do_sample": False}
ms_out = ms_net.generate(**ms_inputs,**gen_kwargs)
outputs = ms_out[:, ms_inputs['input_ids'].shape[1]:]
print(tokenizer.decode(outputs[0]))